In [6]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import time

In [13]:
def initialize(speed = 1, N = 100, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        theta = 2 * np.pi * random()
        print(theta)
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)
        print speeds[i]

    return agents, speeds, plot(agents, width, height)


def plot(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            speeds[i] = speeds[i] + weightedSpeed
            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------


def get_distances(agent, agents):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    
    dists = [(a.getX() - agent.getX())**2 + (a.getY() - agent.getY())**2 for a in agents]
    for i in range(len(dists)):
        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)


def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        print(speeds[i])
        agents[i].move(dxvec[i], dyvec[i])


def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


In [8]:
def couzin(agents, speeds, N, s, rr=1, ro=2, ra=3):
    # watch only particles inrepuls
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents)
        r_dir = np.array([0, 0])
        o_dir = np.array([0, 0])
        a_dir = np.array([0, 0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
                    
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)

        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir 


def vicsek():
    return

def gueron():
    return

In [15]:
def simulate(N_steps, a, dt, N, width, height, s):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """

    agents, speeds, window = initialize(s, N, width, height)
    print(speeds)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        couzin(agents, speeds, N, 10,1900,2000)
        #couple_speeds(agents, speeds, a, s, N)
        treat_boundary(width, height, agents, speeds, N)
        #time.sleep(0.02)
    window.close()

simulate(200, 0.1, 1, 5, 100, 100, 5.0)

3.53129908907
[-4.62510315 -1.89958439]
2.85502595791
[-4.79609993  1.41330302]
6.19249706234
[ 4.97945319 -0.45281994]
1.34347269198
[ 1.12685412  4.87136529]
1.46543220795
[ 0.52584638  4.97227167]
[array([-4.62510315, -1.89958439]), array([-4.79609993,  1.41330302]), array([ 4.97945319, -0.45281994]), array([ 1.12685412,  4.87136529]), array([ 0.52584638,  4.97227167])]
[-4.62510315 -1.89958439]
[-4.79609993  1.41330302]
[ 4.97945319 -0.45281994]
[ 1.12685412  4.87136529]
[ 0.52584638  4.97227167]
[ 8.89378376  4.57171855]
[-0.5325857  -9.98580755]
[ 2.49591819 -9.68351137]
[-7.30032008  6.83412955]
[-2.49591819  9.68351137]
[-9.8242039 -1.8668202]
[ 9.98986638 -0.45007742]
[-6.30176327 -7.76452057]
[-1.74392131  9.84676284]
[-9.2487539   3.80270315]
[ 8.83159989  4.69071886]
[-0.16796478 -9.99858929]
[ 2.48789088 -9.68557685]
[-8.37819234  5.45947736]
[-2.48789088  9.68557685]
[ 8.83159989  4.69071886]
[-0.16796478 -9.99858929]
[ 2.48789088  9.68557685]
[ 8.49351519  5.27827621]
[-